In [1]:
import os
import xml.etree.ElementTree as ET 
import struct
import numpy as np
import cv2
import random
from matplotlib import pyplot as plt
%matplotlib inline
from IPython.core.debugger import Tracer

In [2]:
DATA_DIR = "children_draw"

In [3]:
def parse_xml(image_dir, xml_file):
#     image_dir = ""
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_paths = []
    labels = []

    images = root.find("images")
    images = images.findall("image")
    print("Number of images to parse: " + str(len(images)))

    for idx, image in enumerate(images):
        if idx % 100 == 0:
            print(idx)
        image_name = image.get('file').split('\\')[-1]
        image_path = os.path.join(image_dir, image_name)
        cur_img = cv2.imread(image_path)
        if cur_img == None:
            print("wrong img name: " + xml_file)
            continue
        img_height, img_width, _ = cur_img.shape

        image_labels = []
        boxes = image.findall('box')
        if len(boxes) == 0:
            print("no label data: " + xml_file)
            continue
        for box in boxes:
            obj_label = box.find('label')
            if obj_label.text == "bar":
                continue
            top = int(box.get('top'))
            left = int(box.get('left'))
            width = int(box.get('width'))
            height = int(box.get('height'))

            xmin = np.max([left, 0])
            xmax = np.min([left + width, img_width])
            ymin = np.max([top, 0])
            ymax = np.min([top + height, img_height])

            image_labels.append([xmin, ymin, xmax, ymax, 0])
        
        if len(image_labels) == 0:
            continue

        image_paths.append(os.path.join(image_dir, image_name))
        labels.append(image_labels)
    return image_paths, labels

In [4]:
image_paths = []
labels = []

In [11]:
image_dir = os.path.join(DATA_DIR, "images")
xml_file = os.path.join(DATA_DIR, "images.xml")
image_paths, labels = parse_xml(image_dir, xml_file)

Number of images to parse: 1656
0


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:19: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


In [12]:
def convert_to_string(image_path, labels):
    """convert image_path, lables to string 
    Returns:
    string 
    """
    out_string = ''
    out_string += image_path
    for label in labels:
        for i in label:
            out_string += ' ' + str(i)
    out_string += '\n'
    return out_string

records = []
for idx, image_path in enumerate(image_paths):
    label = labels[idx]
    records.append(convert_to_string(image_path, label))

random.shuffle(records)

In [15]:
val_ratio = 0.1
total_num = len(records)
val_num = int(val_ratio * total_num)
train_num = total_num - val_num
train_records = records[0:train_num]
val_records = records[train_num:]

In [16]:
train_out_file = open("draw_train.txt", "w")
for record in train_records:
    train_out_file.write(record)
train_out_file.close()
val_out_file = open("draw_val.txt", "w")
for record in val_records:
    val_out_file.write(record)
val_out_file.close()